<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/Models/Final%20Models/BiLSTM-CRF/GloVe/BiLSTM-CRF%20(Glove).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install sklearn_crfsuite
!pip install keras==2.2.4
%tensorflow_version 1.x
#!pip install tensorflow-gpu==1.15.0

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-4oxyemca
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-4oxyemca
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=048d08fc815cd318c450b998246792ab411e68f3d585b349476e0ead20d36ed6
  Stored in directory: /tmp/pip-ephem-wheel-cache-hyshfg0m/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
TensorFlow 1.x selected.


In [ ]:
import pandas as pd
import re
import urllib.request
from sklearn.model_selection import train_test_split

#keras and tensorflow packages
import tensorflow as tf
from keras.layers.merge import add
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras_contrib.metrics import crf_accuracy

#evaluation
from sklearn_crfsuite.metrics import flat_classification_report, flat_f1_score, flat_precision_score, flat_recall_score
print(tf.__version__)

1.15.2


Using TensorFlow backend.


In [ ]:
#Ensure Reproducibility
import numpy as np
import random as rn
import os

random_seed=3

# 1) set numpy random seed
np.random.seed(random_seed)
# 2) set build-in random seed
rn.seed(random_seed)
# 3) set PYTHONHASHSEED
os.environ['PYTHONHASHSEED']=str(random_seed)
# 4) set tensorflow random seed
tf.random.set_random_seed(random_seed)

# 5) Force TensorFlow to use single thread.
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Named-Enitty-Recognition/master/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Named-Enitty-Recognition/master/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

def readstring(filename, meth):
    f = filename.split('\n')
    sentences = []
    sentence = []
    for line in f:
        if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == "\n":
            if len(sentence) > 0:
                sentences.append(sentence)
                sentence = []
            continue
        splits = line.split(' ')
        if meth.lower()=="numbers1":
            sentence.append([hasNumbers1(splits[0]), splits[-1].strip()])
        elif meth.lower()=="numbers2":
            sentence.append([hasNumbers2(splits[0]), splits[-1].strip()])
        else:
            sentence.append([splits[0], splits[-1].strip()])
    if len(sentence) > 0:
        sentences.append(sentence)
        sentence = []
    return sentences

def hasNumbers1(inputString):
    if re.search(r'\d', inputString):
        return "0"
    else:return(inputString)

def hasNumbers2(text):
  if text.isdigit():
      return "1"
  elif re.search(r'\d',text) and re.search(r'\,|\.',text):
      return "1" 
  else:
      if re.search(r'\d', text):
          return(re.sub('\d','D', text))
      else:
          return text

#preproces the txt file
train_data = readstring(train,"Numbers1")
test_data = readstring(test,"Numbers1")

#create corpus
corpus = train_data.copy()
corpus.extend(test_data)

In [ ]:
def reformat_data(data,meth):
  if meth.lower() == "data":
    i=0
  else: i=1
  train = []
  output= []
  for sentence in data:
    words=[]
    for x in sentence:
      words.append(x[i])
    train.append(words)

  for i in train:
    string = ' '.join(i)
    output.append(string)
  return output

def get_max_length(corpus):
  length = []
  for sentence in corpus:
    length.append(len(sentence))
  return int(max(length))

def number_of_tags(corpus):
  tags=[]
  for sentence in corpus:
    for tag in sentence:
      tags.append(tag[1])
  return int(len(list(set(tags))))


MAX_LEN = get_max_length(corpus)
N_tags = number_of_tags(corpus)

train = reformat_data(train_data,"data")
test = reformat_data(test_data,"data")


In [ ]:
# create a tokenizer
token_word = text.Tokenizer(char_level=False, lower=True, filters="}", oov_token='UNK')
token_word.fit_on_texts(train)

# convert text to sequence of tokens and pad them to ensure equal length vectors 
X_train = sequence.pad_sequences(token_word.texts_to_sequences(train), maxlen=MAX_LEN,padding="post")
X_test = sequence.pad_sequences(token_word.texts_to_sequences(test), maxlen=MAX_LEN,padding="post")

In [ ]:
train = reformat_data(train_data,"tags")
test = reformat_data(test_data,"tags")

# create a tokenizer
token_tag = text.Tokenizer(char_level=False, lower=False, filters="}")
token_tag.fit_on_texts(train)

# convert text to sequence of tokens and pad them to ensure equal length vectors 
Y = sequence.pad_sequences(token_tag.texts_to_sequences(train), maxlen=MAX_LEN,padding="post")
# One-Hot encode categorical labels
Y_train = [to_categorical(i, num_classes=N_tags+1) for i in Y]

Y = sequence.pad_sequences(token_tag.texts_to_sequences(test), maxlen=MAX_LEN,padding="post")
# One-Hot encode categorical labels
Y_test = [to_categorical(i, num_classes=N_tags+1) for i in Y]

#add padding 
token_tag.index_word[0]="PAD"
sub_label = list(token_tag.index_word.values())
sub_label.remove('O')
sub_label.remove('PAD')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_path = "/content/drive/My Drive/glove.6B.50d.txt"

embeddings_index={}
f = open(root_path, encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
#create emedding matrix
EMBEDDING=int(root_path.split(".")[-2][:-1])
word_index = token_word.word_index
embedding_matrix = np.zeros((len(token_word.word_index) + 1, EMBEDDING))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
      embedding_matrix[i] = embedding_vector

In [ ]:
checks=[]
words=[]
for i in range(0,len(token_word.word_index)+1):
  if embedding_matrix[i][0] == 0.0:
    checks.append(1)
    words.append(list(token_word.word_index.items())[i-1][0])
  else:
    continue

print("Missing words from Embeddings: %d (%.2f%%)" %(len(checks),(len(checks)/len(token_word.word_index)*100)))

Missing words from Embeddings: 1218 (7.10%)


## Create Model

In [ ]:
from keras import optimizers
# Paramters
EPOCHS = 20
BATCH_SIZE = 32
LSTM_CELLS = 300
DROPOUT = 0.25
HIDDEN = 50
CLIP = 1.0

#optimizer
opt=optimizers.Nadam(clipvalue=CLIP)

#define model
input =  Input(shape=(MAX_LEN,))

model_2 = Embedding(len(token_word.word_index) + 1,output_dim=EMBEDDING,
                  weights=[embedding_matrix],input_length=MAX_LEN,
                  trainable=False,mask_zero=True)(input)

model_2 = Bidirectional(LSTM(units=LSTM_CELLS, return_sequences=True,
                           recurrent_dropout=DROPOUT, dropout=DROPOUT))(model_2)

model_2 = TimeDistributed(Dense(HIDDEN, activation="relu"))(model_2)
crf = CRF(N_tags+1)  # CRF layer
out = crf(model_2)  # output
model_2 = Model(input, out)








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
#summary
model_2.compile(optimizer=opt, loss=crf_loss, metrics=[crf_viterbi_accuracy])
model_2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 124)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 124, 50)           858250    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 124, 600)          842400    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 124, 50)           30050     
_________________________________________________________________
crf_1 (CRF)                  (None, 124, 9)            558       
Total params: 1,731,258
Trainable params: 873,008
Non-trainable params: 858,250
_________________________________________________________________


In [ ]:
%%time
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
history = model_2.fit(X_train, np.array(Y_train), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1,callbacks=[early_stopping])



Train on 11232 samples, validate on 2809 samples
Epoch 1/20
11232/11232 [==============================] - 208s 19ms/step - loss: 40.2117 - crf_viterbi_accuracy: 0.9200 - val_loss: 35.2088 - val_crf_viterbi_accuracy: 0.9464
Epoch 2/20
11232/11232 [==============================] - 206s 18ms/step - loss: 40.0727 - crf_viterbi_accuracy: 0.9544 - val_loss: 35.1795 - val_crf_viterbi_accuracy: 0.9544
Epoch 3/20
11232/11232 [==============================] - 206s 18ms/step - loss: 40.0400 - crf_viterbi_accuracy: 0.9635 - val_loss: 35.1603 - val_crf_viterbi_accuracy: 0.9603
Epoch 4/20
11232/11232 [==============================] - 205s 18ms/step - loss: 40.0218 - crf_viterbi_accuracy: 0.9683 - val_loss: 35.1587 - val_crf_viterbi_accuracy: 0.9601
Epoch 5/20
11232/11232 [==============================] - 206s 18ms/step - loss: 40.0106 - crf_viterbi_accuracy: 0.9722 - val_loss: 35.1530 - val_crf_viterbi_accuracy: 0.9629
Epoch 6/20
11232/11232 [==============================] - 205s 18ms/step -

## Predict On Training Set

In [ ]:
%%time
pred = model_2.predict(X_train, verbose=1)

14041/14041 [==============================] - 79s 6ms/step
CPU times: user 2min 7s, sys: 11.6 s, total: 2min 19s
Wall time: 1min 19s
14041/14041 [==============================] - 79s 6ms/step
CPU times: user 2min 7s, sys: 11.9 s, total: 2min 18s
Wall time: 1min 19s


In [ ]:
# TRain Eval
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(Y_train, -1)

# Convert the index to tag
pred_tag = [[token_tag.index_word[i] for i in row] for row in preds]
y_tr_true_tag = [[token_tag.index_word[i] for i in row] for row in y_tr_true]
                 
                 
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=sub_label)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='micro',labels=sub_label)
print(round(score,3))  

              precision    recall  f1-score   support

       I-PER       0.97      0.98      0.98     11128
       I-ORG       0.91      0.89      0.90     10001
       I-LOC       0.97      0.93      0.95      8286
      I-MISC       0.92      0.85      0.88      4556
      B-MISC       1.00      0.14      0.24        37
       B-ORG       1.00      1.00      1.00        24
       B-LOC       1.00      0.82      0.90        11

   micro avg       0.94      0.92      0.93     34043
   macro avg       0.97      0.80      0.83     34043
weighted avg       0.94      0.92      0.93     34043

0.934
              precision    recall  f1-score   support

       I-PER       0.97      0.98      0.98     11128
       I-ORG       0.91      0.89      0.90     10001
       I-LOC       0.97      0.93      0.95      8286
      I-MISC       0.92      0.85      0.88      4556
      B-MISC       1.00      0.14      0.24        37
       B-ORG       1.00      1.00      1.00        24
       B-LOC      

## Predict On Test Set

In [ ]:
%%time
pred = model_2.predict(X_test, verbose=1)

3453/3453 [==============================] - 19s 6ms/step
CPU times: user 31.6 s, sys: 2.91 s, total: 34.5 s
Wall time: 19.5 s
3453/3453 [==============================] - 19s 6ms/step
CPU times: user 31.2 s, sys: 2.99 s, total: 34.2 s
Wall time: 19.5 s


In [ ]:
# Test Eval
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(Y_test, -1)

# Convert the index to tag
pred_tag = [[token_tag.index_word[i] for i in row] for row in preds]
y_tr_true_tag = [[token_tag.index_word[i] for i in row] for row in y_tr_true]
                 
                 
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag, labels=sub_label)
print(report)

f1_score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='micro', labels=sub_label)
recall_score = flat_recall_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='micro', labels=sub_label)
pre_score = flat_precision_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='micro', labels=sub_label)

print("Precision score: %.2f" %round(pre_score*100,3))
print("Recall score: %.2f" %round(recall_score*100,3))
print("F1 score: %.2f" %round(f1_score*100,3))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       I-PER       0.90      0.92      0.91      2773
       I-ORG       0.74      0.76      0.75      2491
       I-LOC       0.90      0.82      0.86      1919
      I-MISC       0.72      0.69      0.71       909
      B-MISC       0.00      0.00      0.00         9
       B-ORG       0.00      0.00      0.00         5
       B-LOC       0.00      0.00      0.00         6

   micro avg       0.83      0.82      0.83      8112
   macro avg       0.47      0.46      0.46      8112
weighted avg       0.83      0.82      0.82      8112

Precision score: 83.12
Recall score: 81.90
F1 score: 82.51


In [ ]:
from sklearn.metrics import confusion_matrix
flat_preds = [item for sublist in pred_tag for item in sublist]
flat_true = [item for sublist in y_tr_true_tag for item in sublist]

sub_label.append("O")
matrix=confusion_matrix(flat_preds,flat_true, labels=sub_label)
pd.DataFrame(matrix, columns = sub_label, index=sub_label)

,I-PER,I-ORG,I-LOC,I-MISC,B-MISC,B-ORG,B-LOC,O
I-PER,2543,74,26,15,0,0,0,168
I-ORG,80,1892,148,72,1,5,3,340
I-LOC,17,91,1583,13,1,0,1,56
I-MISC,0,41,39,626,4,0,0,154
B-MISC,0,0,0,0,0,0,0,0
B-ORG,0,0,0,0,0,0,0,0
B-LOC,0,0,0,0,0,0,0,0
O,133,393,123,183,3,0,2,37605
